# Fixed width file to csv

In this notebook, we first create a Fixed-Width_File (fwf) from an arbitrary dictionary using the configuration given in 'spec.json' file. 

We consider few data corruption scenarios:

1. Length of columns are not equal, i.e. some columns have empty values in some of their cells. This is fixed by fix_dict function.

2. Data type in the cell doens't match the input data in the cell. E.g. the column is supposed to be string (indicated in config), but the input data is integer. Function fix_column_type is responsible for fixint this issue. 

3. Data length is more than what it should be (enforced by config file). If the column is string, we select first n characters. n comes from config file. If it is integer or float, an error is raised as we don't know what could be the correct method to deal with this situation. Another option is to set that cell with corrupted data to a default value (e.g. -1 for int if -1 is not in the range of that column). 

Then the file is written to a fw file named 'new_file.txt' with proper encoding. 

Next, the fwf is opened again, and is read line by line. Each line is delimited by a delimiter (which is a comma by default)  and then, it is written to a csv file named 'new_utf8_file.csv'. 

In [1]:
import json

In [2]:
with open ('spec.json','r') as jf:
    specs_dict = json.load(jf)

In [3]:
config_dict = {col_name:{'length': int(length_val), 'type': str,'default_value' : ''} 
               for (col_name,length_val) in zip(specs_dict['ColumnNames'],
                                                specs_dict['Offsets'])}
config_dict

{'f1': {'length': 5, 'type': str, 'default_value': ''},
 'f2': {'length': 12, 'type': str, 'default_value': ''},
 'f3': {'length': 3, 'type': str, 'default_value': ''},
 'f4': {'length': 2, 'type': str, 'default_value': ''},
 'f5': {'length': 13, 'type': str, 'default_value': ''},
 'f6': {'length': 7, 'type': str, 'default_value': ''},
 'f7': {'length': 10, 'type': str, 'default_value': ''},
 'f8': {'length': 13, 'type': str, 'default_value': ''},
 'f9': {'length': 20, 'type': str, 'default_value': ''},
 'f10': {'length': 13, 'type': str, 'default_value': ''}}

In [4]:
my_data2 = {
    'f1': ['’','AB','AAABBB_B'],
    'f2': ['cd','CD','CCC_DDD_D'],
    'f3': ['ef',1234,'EFFF_FFF'],
    'f4': ['gh','GH','GHGHGHGH'],
    'f5': ['ij','IJ','IJIJIJIJ'],
    'f6': ['kl','KL','KL___KL'],
    'f7': ['mn','MN','MN123456MN'],
    'f8': ['op','OP','OPOP'],
    'f9': ['qr','QR','QRqrQRqr'],
    'f10': ['st','ST','STSTstST']
}

In [5]:
def fix_data_type(data_dict, config =config_dict):
    data_dict = fix_cols_type(data_dict, config)
    data_dict = fix_cols_length(data_dict, config)
    return data_dict

# Fix data type

Data might have wrong type and length, we check and fix those issues

In [6]:
def fix_dict(data_dict,config,max_length_data):
    """ This function fills the empty cells (end of the list) with default values from config file"""
    for field_name in data_dict:
        if len(data_dict[field_name])<max_length_data:
            extesion_list = [config[field_name]['default_value'] for i in range(max_length_data-len(data_dict[field_name])) ]
            data_dict[field_name].extend(extesion_list)
            
    return data_dict   

In [7]:
def fix_cols_type(data_dict, config):
    """If col type is string, change the data to string. If int or float, try to convert"""
    for col_name in config:
        if not isinstance(data_dict[col_name],config[col_name]['type']):
            if config[col_name]['type'] is str:
                data_dict[col_name] = str(data_dict[col_name])
            elif config[col_name]['type'] is int:
                try:
                    data_dict[col_name] = int(dat_dict[col_name])
                except:
                    print('{} cant be converted into int format'.format(data_dict[col_name]))
            elif config[col_name]['type'] is float:
                try:
                    data_dict[col_name] = float(data_dict[col_name])
                except:
                    print('{} cant be converted into float format'.format(data_dict[col_name]))
            
    return data_dict

In [8]:
def fix_cols_length(data_dict,config):
    """if length of data in a string-type column is greater than it should be (i.e. =length), pick first 'length' values. 
    If type is boolean, maybe the input is True, or 1 or yes (etc.), then change the output to 1 
    data_dict, contains one series of data. """ 
#     dct = dict.fromkeys([col_name for col_name in config])
#     print(dct)
    for col_name in config:
        if config[col_name]['type'] is str:
            if len(str(data_dict[col_name])) > config[col_name]['length']:
                data_dict[col_name] = data_dict[col_name][:config[col_name]['length'] ] # first length characters
        elif config[col_name]['type'] is bool:
            if data_dict[col_name] in {1,'yes','Yes','YES','True'}:
                data_dict[col_name]=1
            elif data_dict[col_name] in {0,'no','No','NO','False'}:
                data_dict[col_name]=0
            else:
                raise ValueError("{} is not a bolean type).".format(data_dict[col_name], config[col_name]['length']) )
        
        else:
            if len(str(data_dict[col_name])) > config[col_name]['length']:
                raise ValueError("{} cannot fit into {} spaces).".format(data_dict[col_name], config[col_name]['length']))

    return data_dict

In [9]:
test_case_1 = {'f1': 1,'f2': 'ab'}
config_0 = {'f1': {'length': '5', 'type': int},'f2': {'length': '12', 'type': str} }
config_1 = {'f1': {'length': '5', 'type': float},'f2': {'length': '12', 'type': str} }
config_2 = {'f1': {'length': '5', 'type': str},'f2': {'length': '12', 'type': str} }
config_3 = {'f1': {'length': '5', 'type': float},'f2': {'length': '12', 'type': int} }
config_4 = {'f1': {'length': '5', 'type': bool},'f2': {'length': '12', 'type': str} }


In [10]:
print(fix_cols_type(test_case_1,config_0))
print(fix_cols_type(test_case_1,config_1))
print(fix_cols_type(test_case_1,config_2))
print(fix_cols_type(test_case_1,config_3))
print(fix_cols_type(test_case_1,config_4))



{'f1': 1, 'f2': 'ab'}
{'f1': 1.0, 'f2': 'ab'}
{'f1': '1.0', 'f2': 'ab'}
ab cant be converted into int format
{'f1': 1.0, 'f2': 'ab'}
{'f1': 1.0, 'f2': 'ab'}


In [11]:
test_case_2 = {'f1': 111,'f2': 'abcde'}
config_5 = {'f1': {'length': 5, 'type': int},'f2': {'length': 12, 'type': str} }
config_6 = {'f1': {'length': 2, 'type': int},'f2': {'length': 12, 'type': str} }
config_7 = {'f1': {'length': 5, 'type': str},'f2': {'length': 3, 'type': str} }

test_case_3 = {'f1': True,'f2': 'abcde'}
config_8 = {'f1': {'length': 5, 'type': bool},'f2': {'length': 12, 'type': str} }
config_9 = {'f1': {'length': 5, 'type': str},'f2': {'length': 12, 'type': str} }


In [12]:
print(fix_cols_length(test_case_2,config_5))
# print(fix_cols_length(test_case_2,config_6))
print(fix_cols_length(test_case_2,config_7))
print(fix_cols_type(test_case_3,config_8))
print(fix_cols_type(test_case_3,config_9))

{'f1': 111, 'f2': 'abcde'}
{'f1': 111, 'f2': 'abc'}
{'f1': True, 'f2': 'abcde'}
{'f1': 'True', 'f2': 'abcde'}


# write file to fixed-width-file (fwf)

We assume the data we want to write to fwf is stored in a dictioniary of list objects. Each list object, contains 1 or more entries.

In [13]:
def make_new_line(data_dict,config,encoding = 'utf-8'):
    """ Makes new line with provided input (data) using configuration file (config)"""
    frmt = ''
    line = "".join(['{0:{1}}'.format(data_dict[col_name],config[col_name]['length']) 
                    for col_name in config])+'\n'
    
    line = line.encode(encoding = encoding, errors ='replace')
    print(line)
    return line
        

In [14]:
def break_dict(data_dict,config):
    # when there are more than one entry for each column in the original dictionary
    # some of fields might be empty, find the maximum length
    max_length_data = 0
    for field_name in data_dict:
        max_length_data = max(len(data_dict[field_name]),max_length_data)
    
    # fill the empty cells with default value from config 
    data_dict = fix_dict(data_dict,config,max_length_data)
    
    nested_dict = dict.fromkeys([f_name for f_name in config])
    new_dict = {str(i):dict(nested_dict) for i in range(max_length_data)}
#     print(new_dict)
    
    for field_name in config:
        for idx, val in  enumerate(data_dict[field_name]):
            new_dict[str(idx)][field_name]= data_dict[field_name][idx]

    for col_name in new_dict:
        print(col_name)
        new_dict[col_name] = fix_data_type(new_dict[col_name],config)
        print(new_dict[col_name])
        
    return new_dict

In [15]:
def write_file(data_dict,config, file_name = 'new_file.txt', encoding = 'cp1252'):
    
    segregated_dict = break_dict(data_dict,config)
    header_line = "".join(['{0:{1}}'.format(col_name,int(config[col_name]['length'])) 
                           for col_name in config])+'\n'
    header_line=header_line.encode(encoding)
    with open(file_name,"wb") as f:
        f.write(header_line)
        for dct in segregated_dict:
            f.write(make_new_line(segregated_dict[dct],config_dict,encoding = encoding))

In [16]:
write_file(my_data2,config_dict)

0
{'f1': '’', 'f2': 'cd', 'f3': 'ef', 'f4': 'gh', 'f5': 'ij', 'f6': 'kl', 'f7': 'mn', 'f8': 'op', 'f9': 'qr', 'f10': 'st'}
1
{'f1': 'AB', 'f2': 'CD', 'f3': '123', 'f4': 'GH', 'f5': 'IJ', 'f6': 'KL', 'f7': 'MN', 'f8': 'OP', 'f9': 'QR', 'f10': 'ST'}
2
{'f1': 'AAABB', 'f2': 'CCC_DDD_D', 'f3': 'EFF', 'f4': 'GH', 'f5': 'IJIJIJIJ', 'f6': 'KL___KL', 'f7': 'MN123456MN', 'f8': 'OPOP', 'f9': 'QRqrQRqr', 'f10': 'STSTstST'}
b'\x92    cd          ef ghij           kl     mn        op           qr                  st           \n'
b'AB   CD          123GHIJ           KL     MN        OP           QR                  ST           \n'
b'AAABBCCC_DDD_D   EFFGHIJIJIJIJ     KL___KLMN123456MNOPOP         QRqrQRqr            STSTstST     \n'


# Parsing and Converting to CSV

First we open the fwf and read one line. Then we pass the line to chopper to slice the line based on the config file, then the list from chopper is passed down to joiner, which join the cells with delimiter. The new file is written to a (opened) csv file. 

To avoid memory overload, we read/write line by line (instead of dumping the whole file into local memory). 

In [17]:
def chopper(str_obj, config_dict):
    position = 0
    chopped_string = []
    for config in config_dict.values():
        length = int(config['length'])
        chopped_string.append(str_obj[position:position + length])
        position += length
    return chopped_string

In [18]:
def joiner(chopped_line,delimitor=','):
    return delimitor.join(chopped_line)+'\n'

In [19]:
def fwf_2_csv_line(line,config=config_dict,delimitor = ','):
    chopped_line = chopper(line,config)
    joined_line = joiner(chopped_line,delimitor)
    print(joined_line)
    return joined_line

In [20]:
# open files, read it, write to new file
f_in = open('new_file.txt','r',encoding='cp1252')
f_in_2=open('new_file.txt','r',encoding='utf-8',errors='replace')
f_out = open('new_utf8_file.csv','w',encoding='utf-8')

# Showing that with wrong encoding, the file can't be read
print('file is encoded in cp1252, but if we wrongfully use utf-8, we see ? for unknown characteres\n')
for line in f_in_2:
    print(line)

print('with correct encoding: \n')
for line in f_in:
    comma_joined_line = fwf_2_csv_line(line)
    f_out.write(comma_joined_line)
f_in.close()
f_in_2.close()
f_out.close()

file is encoded in cp1252, but if we wrongfully use utf-8, we see ? for unknown characteres

f1   f2          f3 f4f5           f6     f7        f8           f9                  f10          

�    cd          ef ghij           kl     mn        op           qr                  st           

AB   CD          123GHIJ           KL     MN        OP           QR                  ST           

AAABBCCC_DDD_D   EFFGHIJIJIJIJ     KL___KLMN123456MNOPOP         QRqrQRqr            STSTstST     

with correct encoding: 

f1   ,f2          ,f3 ,f4,f5           ,f6     ,f7        ,f8           ,f9                  ,f10          

’    ,cd          ,ef ,gh,ij           ,kl     ,mn        ,op           ,qr                  ,st           

AB   ,CD          ,123,GH,IJ           ,KL     ,MN        ,OP           ,QR                  ,ST           

AAABB,CCC_DDD_D   ,EFF,GH,IJIJIJIJ     ,KL___KL,MN123456MN,OPOP         ,QRqrQRqr            ,STSTstST     



In [21]:
# The csv file is encoded in utf-8
with open('new_utf8_file.csv','r',encoding='utf-8') as f:
    for line in f:
        print(line)

f1   ,f2          ,f3 ,f4,f5           ,f6     ,f7        ,f8           ,f9                  ,f10          

’    ,cd          ,ef ,gh,ij           ,kl     ,mn        ,op           ,qr                  ,st           

AB   ,CD          ,123,GH,IJ           ,KL     ,MN        ,OP           ,QR                  ,ST           

AAABB,CCC_DDD_D   ,EFF,GH,IJIJIJIJ     ,KL___KL,MN123456MN,OPOP         ,QRqrQRqr            ,STSTstST     



# Notes

Because the file is encoded with utf-8 and Microsoft Excel uses cp1252, it might not show the file correctly for some of the entries (not normal characters, but the ones that binary encoding in utf-8 and cp1252 are different like ' ’ ').

This code works fine with small objects, but for bigger files, the process of reading file line by line and writting to new file may be inconviniently time consuming. For such files, it's better to use PySpark to use the multiprocessing capabilities. 

Because no external libraries has been used for this code, using docker didn't seem to be of any help.